# Lowlevel TensorFlow Functions
This notebook tests a rebuild of NetworkBuilder and additional supporting classes that utilize the lower level TensorFlow API (tf.nn) instead of higher level API (tf.contrib.layers).

## Initial setup (borrowed from Network Visualization notebook)

In [1]:
# If using one or multiple GPUs
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [2]:
import sys
sys.path.insert(0, "../python")
from vizdoom import *
from helper import create_network, create_agent
import tensorflow as tf
%load_ext autoreload
%autoreload 2

In [3]:
# Credit: https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter/38192374#38192374
from IPython.display import clear_output, Image, display, HTML
import numpy as np

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

## layers.py

### On layers branch

In [12]:
phi = 4
channels= 3
num_actions = 4
learning_rate = 0.00025
params_file_path = None
output_directory = "./lowlevel_tf/"
scope = ""

In [4]:
tf.reset_default_graph()
sess = tf.Session()
net_file = "../networks/test.json"
network = create_network(net_file,
                         phi=phi, 
                         num_channels=channels, 
                         num_actions=num_actions,
                         learning_rate=learning_rate,
                         params_file=params_file_path,
                         output_directory=output_directory,
                         session=sess,
                         scope=scope)
show_graph(tf.get_default_graph().as_graph_def())

NHWC
[8, 8, 12, 32]
NHWC
[4, 4, 32, 64]


In [7]:
agent_file_path = "../agents/test.json"
params_file_path = "./lowlevel_tf/dqn_params/train_model-100"
config_file_path = "../config/linear_track.cfg"
results_dir = "../tmp/tmp_results/"
action_set = "basic_four"

### On layers branch

In [8]:
# Initializes DoomGame from config file
def initialize_vizdoom(config_file):
    print("Initializing doom... ", end=""), sys.stdout.flush()
    game = DoomGame()
    game.load_config(config_file)
    game.set_window_visible(True)
    game.init()
    print("Done.")
    return game  

In [10]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))
a_best = agent.network.get_best_action(agent.state).item()
print("a_best: ", a_best)
game.close()

Initializing doom... Done.
NHWC
[8, 8, 12, 32]
NHWC
[4, 4, 32, 64]
INFO:tensorflow:Restoring parameters from ./lowlevel_tf/dqn_params/train_model-100
Q:  [[-3.93629551 -3.95156717 -3.94152212 -3.9497273 ]]
a_best:  0


### On develop branch

In [25]:
phi = 4
channels= 3
num_actions = 4
learning_rate = 0.00025
params_file_path = None
output_directory = "./lowlevel_tf/"
scope = "main_network"

In [26]:
tf.reset_default_graph()
sess = tf.Session()
net_file = "../networks/test.json"
network = create_network(net_file,
                         phi=phi, 
                         num_channels=channels, 
                         num_actions=num_actions,
                         learning_rate=learning_rate,
                         params_file=params_file_path,
                         output_directory=output_directory,
                         session=sess,
                         scope=scope)
show_graph(tf.get_default_graph().as_graph_def())

In [27]:
agent_file_path = "../agents/test.json"
params_file_path = "./lowlevel_tf/dqn_params/train_model-100"
config_file_path = "../config/linear_track.cfg"
results_dir = "../tmp/tmp_results/"
action_set = "basic_four"

In [28]:
# Initializes DoomGame from config file
def initialize_vizdoom(config_file):
    print("Initializing doom... ", end=""), sys.stdout.flush()
    game = DoomGame()
    game.load_config(config_file)
    game.set_window_visible(True)
    game.init()
    print("Done.")
    return game  

In [29]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))
a_best = agent.network.get_best_action(agent.state).item()
print("a_best: ", a_best)
game.close()

Initializing doom... Done.
INFO:tensorflow:Restoring parameters from ./lowlevel_tf/dqn_params/train_model-100
Q:  [[-3.93375802 -3.94898081 -3.93895984 -3.94714355]]
a_best:  0


In [23]:
tf.reset_default_graph()
game = initialize_vizdoom(config_file_path)
agent = create_agent(agent_file_path,
                     game=game, 
                     params_file=params_file_path,
                     action_set=action_set,
                     output_directory=results_dir,
                     train_mode=False)
agent.initialize_new_episode()
current_screen = game.get_state().screen_buffer
agent.update_state(current_screen)
print("Q: ", agent.network.get_q_values(agent.state))
print("Q_bias: ", agent.sess.run(tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)[-1]))
a_best = agent.network.get_best_action(agent.state).item()
print("a_best: ", a_best)
game.close()

Initializing doom... Done.
INFO:tensorflow:Restoring parameters from ./lowlevel_tf/dqn_params/train_model-100
Q:  [[ 0.  0.  0.  0.]]
Q_bias:  [-3.08783484 -3.08685422 -3.08485365 -3.08579946]
a_best:  0


Calculations appear the same on both branches (with DQN).

### Implementing batch norm layer

In [5]:
phi = 4
channels= 3
num_actions = 4
learning_rate = 0.00025
params_file_path = None
output_directory = "./lowlevel_tf/"
scope = "main_network"

In [11]:
tf.reset_default_graph()
sess = tf.Session()
net_file = "../networks/test.json"
network = create_network(net_file,
                         phi=phi, 
                         num_channels=channels, 
                         num_actions=num_actions,
                         learning_rate=learning_rate,
                         params_file=params_file_path,
                         output_directory=output_directory,
                         session=sess,
                         scope=scope)
show_graph(tf.get_default_graph().as_graph_def())

Tensor("main_network/CONV_1/convolution:0", shape=(?, 32, 20, 20), dtype=float32)


TypeError: 'int' object is not iterable